<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/221107_extra_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
import csv
import re
import gzip
import shutil

In [26]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [27]:
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)
urine = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/urine_output_rate.csv', low_memory=False, dtype='str', keep_default_na=False)
age = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/age.csv', low_memory=False, dtype='str', keep_default_na=False)
weight = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/weight_durations.csv', low_memory=False, dtype='str', keep_default_na=False)
vaso=pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/vasoactive_agent.csv', low_memory=False, dtype='str', keep_default_na=False)
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)

In [28]:
# vaso

In [35]:
admission = pd.merge(left = admission, right = sepsis[['subject_id','stay_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
urine = pd.merge(left = urine, right = admission[['subject_id', 'hadm_id','stay_id']], how='inner', left_on='stay_id', right_on='stay_id')
age = pd.merge(left = age, right = admission[['subject_id', 'hadm_id','stay_id']], how='inner', left_on = ['hadm_id','subject_id'], right_on = ['hadm_id', 'subject_id'])


Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'stay_id', 'sepsis3'],
      dtype='object')

In [45]:
urine=urine[['subject_id', 'hadm_id', 'stay_id', 'weight', 'urineoutput_6hr', 'urineoutput_12hr', 'urineoutput_24hr']]

In [50]:
extra_feature = pd.merge(left = urine, right = age[['subject_id', 'hadm_id','stay_id','age']], how='inner', left_on=['subject_id', 'hadm_id', 'stay_id'], right_on=['subject_id', 'hadm_id', 'stay_id'])

In [51]:
extra_feature

,subject_id,hadm_id,stay_id,weight,urineoutput_6hr,urineoutput_12hr,urineoutput_24hr,age
0,11861582,27370071,35689140,89,271,971,971,53
1,11861582,27370071,35689140,89,913,913,913,53
2,11861582,27370071,35689140,89,700,700,700,53
3,11861582,27370071,35689140,89,286,986,986,53
4,11861582,27370071,35689140,89,191,1001,1001,53
...,...,...,...,...,...,...,...,...
11715329,18230098,28932526,39012076,78.3,30,30,30,93
11715330,18230098,29218638,39012076,78.3,30,30,30,88
11715331,18230098,29464315,39012076,78.3,30,30,30,91
11715332,18230098,29882068,39012076,78.3,30,30,30,89
